<a href="https://colab.research.google.com/github/jrmui/jjj/blob/main/J%5E3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%load_ext cudf.pandas

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn as sk
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import mean_squared_error
import tensorflow as tf
import matplotlib.pyplot as plt
import warnings
import os
import zipfile

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_subset = pd.read_csv('/content/drive/My Drive/hackathon data/train.csv', nrows = 100000)

In [ ]:
full_data = pd.read_csv('/content/drive/My Drive/hackathon data/train.csv')

In [ ]:
test_data = pd.read_csv('/content/drive/My Drive/hackathon data/test.csv')

In [ ]:
def random_impute(col):
    # Replace -1 with np.nan to handle both as missing values
    col = col.replace(-1, np.nan)

    # Get only the valid (non-missing) values
    valid_values = col.dropna()

    # Check if valid_values is not empty
    if valid_values.empty:
        # If no valid values, return the column unchanged or fill with a default value
        return col  # or you can fill with a specific value, e.g., col.fillna(0)

    # Impute missing values by sampling from valid values
    col = col.fillna(pd.Series(np.random.choice(valid_values, size=col.isna().sum()), index=col.index[col.isna()]))

    return col


In [ ]:
def standardize(col):
  return (col - np.mean(col)) / np.std(col)

In [ ]:
cleaned_full = full_data.apply(random_impute, axis = 0)
cleaned_full = cleaned_full[(cleaned_full['y'] >= 36000) & (cleaned_full['y'] <= 48000)]
X_cleaned_full = cleaned_full.drop(columns = ['y', 'trickortreat', 'kingofhalloween'], axis = 1)
X_cleaned_full = X_cleaned_full.apply(standardize, axis = 0)
y_cleaned_full = cleaned_full['y']

In [ ]:
cleaned = data_subset.apply(random_impute, axis = 0)
cleaned = cleaned[(cleaned['y'] >= 38000) & (cleaned['y'] <= 46000)]
X_cleaned = cleaned.drop(columns = ['y', 'trickortreat', 'kingofhalloween'], axis = 1)
X_cleaned = X_cleaned.apply(standardize, axis = 0)
y_cleaned = cleaned['y']

In [ ]:
features = ['noon', 'confectionary', 'jacksantino', 'mystical', 'paranormal',
       'quirky', 'cobwebs', 'magical', 'h2o', 'mummy', 'wardrobe', 'moonlight',
       'moon', 'worktime', 'haunt', 'scoobydoo', 'somerset', 'dracula',
       'divination', 'kidnap', 'periodoftime', 'dungeon', 'toys', 'lunacy',
       'casket', 'fire', 'monsterhunter', 'tabulatable']

In [ ]:
X_cleaned_full = X_cleaned_full[features]

In [ ]:
X_cleaned_full_stardardized = X_cleaned_full.apply(standardize, axis = 0)

In [ ]:
elastic_net = ElasticNet(fit_intercept = True)

param_grid = {
    'alpha': [2.66156, 2.66157, 2.66158, 2.66159, 2.6616, 2.66161, 2.66162, 2.66163, 2.66164, 2.66165],       # Regularization strength
    'l1_ratio': [1]    # Balance between L1 and L2
}

# Set up GridSearchCV with 5-fold cross-validation
grid_search = GridSearchCV(estimator=elastic_net, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_cleaned, y_cleaned)

# Best parameters from the grid search
best_alpha = grid_search.best_params_['alpha']
best_l1_ratio = grid_search.best_params_['l1_ratio']

print(f"Best Alpha: {best_alpha}")
print(f"Best L1 Ratio: {best_l1_ratio}")


Best Alpha: 2.66156
Best L1 Ratio: 1


In [ ]:
lasso_full = Lasso(alpha = 2.66156, fit_intercept = True)
lasso_full.fit(X_cleaned_full[features], y_cleaned_full)

Lasso(alpha=2.66156)

In [ ]:
cleaned_test = test_data[features].apply(random_impute, axis = 0).apply(standardize, axis = 0)

In [ ]:
test_prediction = lasso_full.predict(cleaned_test)

In [ ]:
test_data['y'] = test_prediction

In [ ]:
df = test_data[['id', 'y']]

In [ ]:
df.head()

,id,y
0,12000000,42214.963100
1,12000001,42379.123437
2,12000002,42400.408172
3,12000003,42296.629504
4,12000004,42519.474801


In [ ]:
df.to_csv('submission4.csv', index = False)